<img align="left" src="https://panoptes-uploads.zooniverse.org/project_avatar/86c23ca7-bbaa-4e84-8d8a-876819551431.png" type="image/png" height=100 width=100>
</img>
<h1 align="right">KSO Tutorials #10: Run machine learning models on footage</h1>
<h3 align="right"><a href="https://colab.research.google.com/github/ocean-data-factory-sweden/kso/blob/master/tutorials/10_Run_ML_Models_on_footage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a></h3>
<h3 align="right">Written by the KSO Team</h3>

# Set up and requirements

### Install all the requirements

If you are running this notebook in Google Colab, this cell should take ~2 mins and might restart the session. Please run this cell until you get the "Successful installation!" message.

In [ ]:
import sys
import os

# Check if notebook is running in colab
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    # Clone kso repo and install requirements
    if not os.path.exists("kso"):
        print("Installing all dependencies...")
        !git clone https://github.com/ocean-data-factory-sweden/kso.git
        !pip install -r /content/kso/requirements_colab.txt

    # Enable external widgets and navigate to the kso tutorial folder
    try:
        from google.colab import output

        output.enable_custom_widget_manager()
        os.chdir("kso/tutorials")
    except ImportError:
        pass

# Prepare the dev settings if needed
try:
    if "kso_utils" not in sys.modules:
        sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))
        import kso_utils

        print("Using development version...")
        # Enables testing changes in utils
        %load_ext autoreload
        %autoreload 2
except ImportError:
    print("Installing latest version from PyPI...")
    %pip install -q kso-utils

# Avoid issues with widgets not displaying properly
!jupyter nbextension enable --user --py widgetsnbextension
!jupyter nbextension enable --user --py jupyter_bbox_widget
!jupyter nbextension enable --user --py ipysheet

# Load the clear output function to keep things clean
from IPython.display import clear_output

clear_output()
print("Successful installation... you're good to go!")

### Import Python packages

In [ ]:
# Import required modules for tut#10
import kso_utils.widgets as kso_widgets
import kso_utils.project_utils as p_utils
import kso_utils.tutorials_utils as t_utils
import kso_utils.yolo_utils as y_utils
from kso_utils.project import ProjectProcessor, MLProjectProcessor
import os

print("Packages loaded successfully")

### Choose your project

In [ ]:
project_name = kso_widgets.choose_project()

### Initiate project's database

In [ ]:
# Save the name of the project
project = p_utils.find_project(project_name=project_name.value)
# Initiate pp
pp = ProjectProcessor(project)

In [ ]:
# Initialise mlp
mlp = MLProjectProcessor(pp)

# Run model on custom footage

### Choose the model

In [ ]:
model = mlp.choose_model()

### Choose folder to download the model

In [ ]:
download_dir = kso_widgets.choose_folder(".", "where to download the model")

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Download model</font> { vertical-output: true }
artifact_dir = mlp.get_model(model.value, download_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵឵឵<i>Choose your custom footage</font> { vertical-output: true }
source = kso_widgets.choose_footage(
    project=pp.project,
    server_connection=pp.server_connection,
    db_connection=pp.db_connection,
    start_path=pp.project.movie_folder
    if pp.project.movie_folder not in [None, "None"]
    else ".",
    folder_type="custom footage",
)

In [ ]:
source_value = t_utils.process_source(source)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder where to save the runs</font> { vertical-output: true }
# This should be left as default value in most cases.
save_dir = kso_widgets.choose_folder(".", "runs output")

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose a confidence threshold for evaluation</font> { vertical-output: true }
conf_thres = t_utils.choose_conf()

In [ ]:
# @title <font size="5">↓ ឵឵<i>Run model over selected custom footage</font> { vertical-output: true }
mlp.detect_yolov5(
    source=source_value,
    save_dir=save_dir.selected,
    conf_thres=conf_thres.value,
    artifact_dir=artifact_dir,
    exp_name="detect",
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Choose folder with the evaluation data</font> { vertical-output: true }
eval_dir = kso_widgets.choose_folder(
    save_dir.selected
    if "save_dir" in vars() and save_dir.selected is not None
    else ".",
    "runs output",
)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Add the data to Weights and Biases</font> { vertical-output: true }
mlp.save_detections(conf_thres.value, model.value, eval_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵<i>View model output</font> { vertical-output: true }
viewer = y_utils.choose_files(eval_dir.selected)

In [ ]:
# @title <font size="5">↓ ឵឵<i>Investigate training and validation datasets (only image data)</font> { vertical-output: true }
train_dataset, val_dataset = mlp.get_dataset(model.value)

In [ ]:
# Training set
y_utils.get_data_viewer(os.path.join(train_dataset, "data/images"))

In [ ]:
# Validation set
y_utils.get_data_viewer(os.path.join(val_dataset, "data/images"))

# Track unique individuals (Optional)

In [ ]:
mlp.track_individuals(
    source=source_value,
    artifact_dir=artifact_dir,
    eval_dir=eval_dir.selected,
    conf_thres=conf_thres.value,
    img_size=(540, 540),
    name="track",
)

In [ ]:
# END